In [9]:
import itertools
import json
import math
import os
import random
import re
import sys
from collections import Counter
from collections import defaultdict
from datetime import datetime

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from tqdm import tqdm
from utils import *

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [10]:
activity_learn_log = pd.read_csv('./data/activity_learn_log.csv')

In [11]:
activity_learn_log['event'].unique()

array(['start', 'doing', 'watching', 'finish'], dtype=object)

In [12]:
activity_learn_log = activity_learn_log[~(activity_learn_log["event"].isin(["start", "finish"]))]

In [22]:
json_data = {}
# json_data["userId_list"] = sorted(activity_learn_log["userId"]..astype(np.int32))

def calc(df: pd.DataFrame):
    dicts = count_pair(df[["mediaType", "learnedTime"]].values)
    return dicts
    return {"watching": dicts.get("video", 0), "doing": sum(dicts.values()) - dicts.get("video", 0)}

json_data["user_learning_type"] = (
    activity_learn_log[["userId", "mediaType", "learnedTime"]].groupby("userId").apply(calc).to_dict()
)

# json_data["user_learning_record"] = (
#     activity_learn_log[["userId", "learnedTime"]].groupby("userId").agg(list).to_dict()["learnedTime"]
# )
# json_data["user_learning_predict"] = {}

In [23]:
json_data["user_learning_type"]

{3: {'video': 29},
 5: {'download': 113, 'testpaper': 1136, 'video': 10806},
 13: {'video': 15465},
 14: {'download': 4, 'testpaper': 5503},
 16: {'download': 15355, 'video': 33595},
 21: {'video': 97200},
 22: {'testpaper': 577},
 24: {'video': 383},
 98: {'download': 3751, 'video': 33444},
 129: {'download': 6, 'video': 9212},
 153: {'testpaper': 4533, 'video': 2153},
 404: {'video': 621},
 581: {'video': 8234},
 704: {'video': 918},
 705: {'homework': 62},
 877: {'download': 5072, 'video': 61594},
 1057: {'download': 717, 'video': 1407},
 1089: {'video': 1545},
 1119: {'download': 1036, 'video': 1305},
 1168: {'download': 11, 'testpaper': 115824, 'text': 9, 'video': 52678},
 1192: {'video': 40429},
 1272: {'video': 17270},
 1316: {'video': 115},
 1385: {'download': 7486, 'ppt': 9, 'text': 22, 'video': 137438},
 1533: {'download': 372},
 1904: {'download': 1372, 'testpaper': 14799, 'video': 310773},
 2094: {'download': 2919, 'video': 18182},
 2200: {'download': 166, 'homework': 121, 

In [24]:
jsondata = json.load(open('./out/用户学习时长情况与预测.json'))

In [25]:
json_data['user_learning_type']

{3: {'video': 29},
 5: {'download': 113, 'testpaper': 1136, 'video': 10806},
 13: {'video': 15465},
 14: {'download': 4, 'testpaper': 5503},
 16: {'download': 15355, 'video': 33595},
 21: {'video': 97200},
 22: {'testpaper': 577},
 24: {'video': 383},
 98: {'download': 3751, 'video': 33444},
 129: {'download': 6, 'video': 9212},
 153: {'testpaper': 4533, 'video': 2153},
 404: {'video': 621},
 581: {'video': 8234},
 704: {'video': 918},
 705: {'homework': 62},
 877: {'download': 5072, 'video': 61594},
 1057: {'download': 717, 'video': 1407},
 1089: {'video': 1545},
 1119: {'download': 1036, 'video': 1305},
 1168: {'download': 11, 'testpaper': 115824, 'text': 9, 'video': 52678},
 1192: {'video': 40429},
 1272: {'video': 17270},
 1316: {'video': 115},
 1385: {'download': 7486, 'ppt': 9, 'text': 22, 'video': 137438},
 1533: {'download': 372},
 1904: {'download': 1372, 'testpaper': 14799, 'video': 310773},
 2094: {'download': 2919, 'video': 18182},
 2200: {'download': 166, 'homework': 121, 

In [26]:
jsondata['user_learning_type'] = json_data["user_learning_type"]

In [27]:
json.dump(jsondata, open("./out/用户学习时长情况与预测.json", "w"))

In [ ]:
json_data

In [ ]:
json_data.keys()

In [ ]:
def predict(userId):
    data = json_data["user_learning_record"][userId]
    model = ARIMA(data, order=(1, 0, 1))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=10)
    return list(forecast)
for userId in tqdm(json_data["userId_list"]):
    if len(json_data['user_learning_record'][userId]) >= 3:
            json_data["user_learning_predict"][userId] = predict(userId)
    else:
        json_data["user_learning_predict"][userId] = json_data["user_learning_record"][userId]

In [ ]:
for userId in json_data["userId_list"]:
    ls = json_data['user_learning_predict'][userId]
    json_data['user_learning_predict'][userId] = [float(f'{float(x):.3f}') for x in ls]

In [ ]:
json.dump(json_data, open("./out/user_learning_statistics.json", "w"))